<a href="https://colab.research.google.com/github/Aaliya-feisal/Weights_and_biases/blob/main/phishingurl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phishing URL Detection Project

This project aims to detect phishing URLs using logistic regression and decision tree models. We will also integrate Weights & Biases (W&B) for experiment tracking.

## 1. Install Necessary Libraries

First, we need to install the required libraries.


In [2]:
!pip install pandas scikit-learn wandb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import wandb



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


## 3. Initialize Weights & Biases (W&B) Project

We initialize a new W&B project to track our experiments.


In [4]:
# Initialize W&B project
wandb.init(project="phishing-url-detection")


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
#load dataset
#seems like the datset cant use UTF8 coding so we introduce another one
#The CSV file had rows with inconsistent numbers of fields, causing a ParserError. so iUsed the error_bad_lines parameter to skip problematic lines
url_data = pd.read_csv('/content/urlset.csv', encoding='ISO-8859-1', on_bad_lines='skip')




<ipython-input-10-031f24b1fbc6>:4: DtypeWarning: Columns (1,2,3,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  url_data = pd.read_csv('/content/urlset.csv', encoding='ISO-8859-1', on_bad_lines='skip')


In [11]:
url_data.head()

,domain,ranking,mld_res,mld.ps_res,card_rem,ratio_Rrem,ratio_Arem,jaccard_RR,jaccard_RA,jaccard_AR,jaccard_AA,jaccard_ARrd,jaccard_ARrem,label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,10000000,1.0,0.0,18.0,107.611111,107.277778,0.0,0.0,0.0,0.0,0.8,0.795729,1.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,10000000,0.0,0.0,11.0,150.636364,152.272727,0.0,0.0,0.0,0.0,0,0.768577,1.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,10000000,0.0,0.0,14.0,73.500000,72.642857,0.0,0.0,0.0,0.0,0,0.726582,1.0
3,mail.printakid.com/www.online.americanexpress....,10000000,0.0,0.0,6.0,562.000000,590.666667,0.0,0.0,0.0,0.0,0,0.85964,1.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,10000000,0.0,0.0,8.0,29.000000,24.125000,0.0,0.0,0.0,0.0,0,0.748971,1.0


In [12]:
#data descreption
url_data.describe()

,card_rem,ratio_Rrem,ratio_Arem,jaccard_RR,jaccard_RA,jaccard_AR,jaccard_AA,label
count,95923.000000,95923.000000,95923.000000,95922.000000,95921.000000,95920.000000,95919.000000,95913.000000
mean,4.580498,135.255201,138.544211,0.005030,0.003787,0.003378,0.003661,0.499453
std,4.466073,160.988895,175.480485,0.311308,0.024815,0.024011,0.028492,0.500002
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,43.000000,39.666667,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,104.000000,103.333333,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.000000,174.142857,178.292857,0.000000,0.000000,0.000000,0.000000,1.000000
max,187.333333,5507.000000,6097.000000,96.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
url_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96005 entries, 0 to 96004
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   domain         96005 non-null  object 
 1   ranking        95953 non-null  object 
 2   mld_res        95935 non-null  object 
 3   mld.ps_res     95924 non-null  object 
 4   card_rem       95923 non-null  float64
 5   ratio_Rrem     95923 non-null  float64
 6   ratio_Arem     95923 non-null  float64
 7   jaccard_RR     95922 non-null  float64
 8   jaccard_RA     95921 non-null  float64
 9   jaccard_AR     95920 non-null  float64
 10  jaccard_AA     95919 non-null  float64
 11  jaccard_ARrd   95919 non-null  object 
 12  jaccard_ARrem  95917 non-null  object 
 13  label          95913 non-null  float64
dtypes: float64(8), object(6)
memory usage: 10.3+ MB


In [14]:
#check the missing data in each column
url_data.isnull().sum()

domain            0
ranking          52
mld_res          70
mld.ps_res       81
card_rem         82
ratio_Rrem       82
ratio_Arem       82
jaccard_RR       83
jaccard_RA       84
jaccard_AR       85
jaccard_AA       86
jaccard_ARrd     86
jaccard_ARrem    88
label            92
dtype: int64

In [15]:
#seems like ther eis alot of missing data so we will drop those rows
#drop the mssing data
url_data = url_data.dropna()

In [16]:
url_data.isnull().sum()

domain           0
ranking          0
mld_res          0
mld.ps_res       0
card_rem         0
ratio_Rrem       0
ratio_Arem       0
jaccard_RR       0
jaccard_RA       0
jaccard_AR       0
jaccard_AA       0
jaccard_ARrd     0
jaccard_ARrem    0
label            0
dtype: int64

## 6. Convert Columns with Mixed Data Types to String

We encountered issues with mixed data types in some columns, causing errors during label encoding. Converting these columns to string resolves the issue.


In [18]:
# Ensure all columns with mixed types are converted to string
for column in url_data.columns:
    if url_data[column].dtype == 'object' or url_data[column].apply(type).nunique() > 1:
        url_data[column] = url_data[column].astype(str)

## 7. Convert Categorical Data to Numeric

We use `LabelEncoder` to convert categorical data to numeric values.


In [19]:
# Convert categorical data to numeric
label_encoders = {}
for column in url_data.columns:
    if url_data[column].dtype == 'object':
        le = LabelEncoder()
        url_data[column] = le.fit_transform(url_data[column])
        label_encoders[column] = le


## 8. Feature and Target Separation

We separate the features and the target variable from the dataset.


In [20]:
# Feature and target separation
X = url_data.drop('label', axis=1)
y = url_data['label']


## 9. Split the Data

We split the data into training and testing sets.


In [21]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 10. Train Logistic Regression Model

We train a logistic regression model and calculate its accuracy.


In [23]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)



In [24]:

log_reg.fit(X_train, y_train)



LogisticRegression(max_iter=1000)

In [25]:
log_reg_pred = log_reg.predict(X_test)

In [26]:
log_reg_accuracy = accuracy_score(y_test, log_reg_pred)

## 11. Log Logistic Regression Metrics to W&B

We log the accuracy of the logistic regression model to W&B.


In [27]:
# Log logistic regression metrics to W&B
wandb.log({"log_reg_accuracy": log_reg_accuracy})


## 12. Train Decision Tree Model

We train a decision tree model and calculate its accuracy.


In [28]:
tree_clf = DecisionTreeClassifier()

In [29]:
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [30]:
tree_clf_pred = tree_clf.predict(X_test)

In [31]:
tree_clf_accuracy = accuracy_score(y_test, tree_clf_pred)

## 13. Log Decision Tree Metrics to W&B

We log the accuracy of the decision tree model to W&B.


In [32]:
# Log decision tree metrics to W&B
wandb.log({"tree_clf_accuracy": tree_clf_accuracy})


In [33]:
# Finish the W&B run
wandb.finish()


log_reg_accuracy,▁
tree_clf_accuracy,▁
log_reg_accuracy,0.81306
tree_clf_accuracy,0.94938
